In [15]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import sys
sys.path.append("./2018-NEON-beetles-processing/segmentation") 
from predict_masks import *


from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

In [16]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
        # point_coords = np.array(ann['point_coords'])
        # ax.plot(point_coords[:, 0], point_coords[:, 1], 'o', color='red', markersize=5)

    ax.imshow(img)

In [ ]:

def crop_to_bb(img):
    labels = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) > 10
    where = np.nonzero(labels)
    y1, x1 = np.amin(where, axis=1)
    y2, x2 = np.amax(where, axis=1)
    img_cropped = img[y1:y2,x1:x2, :]
    return img_cropped


def is_mask_closer_to_white(image, mask):
    """Check if the masked region of the image is closer to white than to black."""
    masked_pixels = image[mask > 0]
    average_color = np.mean(masked_pixels, axis=0)
    
    white = np.array([255, 255, 255])
    black = np.array([0, 0, 0])
    
    distance_to_white = np.linalg.norm(average_color - white)
    distance_to_black = np.linalg.norm(average_color - black)
    
    return distance_to_white < distance_to_black

def save_anns_elythra(anns, image, path):
    max_area = 0
    best_res = None    

    for i, ann in enumerate(anns):
        # print(f"Processing annotation {i}")
        mask = ann['segmentation']
        res = np.asarray(image) * np.repeat(mask[:, :, None], 3, axis=2)

        # Check if the masked region is closer to white than to black, we filter the background regions
        if is_mask_closer_to_white(image, mask):
            continue

        # Crop to bounding box
        res_cropped = crop_to_bb(res)

        # Calculate area of segmentation
        area = np.sum(mask)

        # Check if this is the largest segmentation found so far
        if area > max_area:
            max_area = area
            best_res = res

    # Save the largest non-background segmentation
    if best_res is not None:
        # print(f"Saving the largest segmentation to {path}")
        best_res_bgr = cv2.cvtColor(best_res, cv2.COLOR_RGB2BGR)

        # Now save the image with the correct color channels
        cv2.imwrite(path, best_res_bgr)
        # plt.figure(figsize=(20,20))
        # plt.imshow(best_res)
        # plt.axis('off')
        # plt.show() 
    else:
        print("No valid segmentation found to save.")



In [26]:
dataset_folder = '../../shared/2018-NEON-beetles/separate_segmented_beetle_images'
output_folder = './data/elythra_segmented_full/'


In [27]:
image_filepaths = read_image_paths(dataset_folder)
print(f'Number of images in dataset: {len(image_filepaths)}')

Number of images in dataset: 0


In [28]:
file_path = './results__px_cm.csv'
import csv

# Initialize an empty list to store the file contents
px_data = []

# Open and read the file
with open(file_path, mode='r') as file:
    reader = csv.reader(file)
    
    # Skip the header
    next(reader)
    
    # Read each row and store as a tuple in the list
    for row in reader:
        image = row[0].strip()  # Assuming the image name is in the second column
        px_to_cm = float(row[1].strip())  # Assuming the px_to_cm value is in the third column
        px_data.append((image, px_to_cm))

print(px_data[0])

('A00000034558.jpg', 258.0)


In [29]:
def get_px_to_cm(px_data, directory):
    for img, val in px_data:
        if img.split('.')[0] == directory:
            return int(val)
    return None

In [30]:


use_cuda = torch.cuda.is_available()
DEVICE   = torch.device("cuda:0" if use_cuda else "cpu")
print("Device: ", DEVICE)

if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)


# load SAM model
segmentation_model = get_sam_model(DEVICE)

mask_generator = SamAutomaticMaskGenerator(model=segmentation_model, box_nms_thresh=0.5)

Device:  cuda:0
__CUDNN VERSION: 90100
__Number CUDA Devices: 1
__CUDA Device Name: GRID A100X-40C
__CUDA Device Total Memory [GB]: 42.945347584


In [32]:
image_extensions = ['.jpg', '.jpeg', '.png', ]
csv_file = './results__segm_elythra_full.csv'

group_image_dirs = [name for name in os.listdir(dataset_folder) 
                    if os.path.isdir(os.path.join(dataset_folder, name))]

image_paths = {}
data = []

def generate_non_overlapping_masks(image, mask_generator):
    # Generate masks for the image
    masks = mask_generator.generate(image)
    
    # Filter masks to ensure they are non-overlapping
    non_overlapping_masks = []
    for mask in masks:
        overlap = False
        for existing_mask in non_overlapping_masks:
            # Check for overlap with existing masks
            if (mask['segmentation'] & existing_mask['segmentation']).any():
                overlap = True
                break
        if not overlap:
            non_overlapping_masks.append(mask)
    
    return non_overlapping_masks

# Iterate through each first-level directory
for directory in group_image_dirs:
    dir_path = os.path.join(dataset_folder, directory)
    images = []
    px_to_cm = get_px_to_cm(px_data, directory)
    if px_to_cm is not None:
        # List all files in the directory and filter out images
        for file in os.listdir(dir_path):
            print(file)
            if any(file.lower().endswith(ext) for ext in image_extensions):
                segmented_path = os.path.join(output_folder, directory, file)
                print(segmented_path, directory)
                data.append((directory, file, px_to_cm))
                fp = os.path.join(dir_path, file)
                image = cv2.imread(fp)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                masks = generate_non_overlapping_masks(image, mask_generator)
                # masks = mask_generator.generate(image)
                segm_folder =  os.path.join(output_folder, directory)
                os.makedirs(segm_folder, exist_ok=True)
                
                #save mask with cv2 to preserve pixel categories
                print(f"Mask path:{segmented_path}")
                save_anns(masks, image, segmented_path)

                with open(csv_file, mode='a', newline='') as f:
                    writer = csv.writer(f)
                    # The header row should only be written once, so ensure it's not duplicated
                    # Check if the file is empty before writing the header (optional)
                    f.seek(0, os.SEEK_END)  # Move to the end of the file
                    if f.tell() == 0:  # If file is empty, write the header
                        writer.writerow(['group_image', 'individual_image', 'px_to_cm'])  
                    writer.writerow((directory, str(file), px_to_cm))
            


    
    

beetle_59.png
./data/elythra_segmented_full/A00000051187/beetle_59.png A00000051187
Mask path:./data/elythra_segmented_full/A00000051187/beetle_59.png
beetle_336.png
./data/elythra_segmented_full/A00000051187/beetle_336.png A00000051187
Mask path:./data/elythra_segmented_full/A00000051187/beetle_336.png
beetle_63.png
./data/elythra_segmented_full/A00000051187/beetle_63.png A00000051187
Mask path:./data/elythra_segmented_full/A00000051187/beetle_63.png
beetle_597.png
./data/elythra_segmented_full/A00000051187/beetle_597.png A00000051187
Mask path:./data/elythra_segmented_full/A00000051187/beetle_597.png
beetle_604.png
./data/elythra_segmented_full/A00000051187/beetle_604.png A00000051187
Mask path:./data/elythra_segmented_full/A00000051187/beetle_604.png
beetle_38.png
./data/elythra_segmented_full/A00000051187/beetle_38.png A00000051187
Mask path:./data/elythra_segmented_full/A00000051187/beetle_38.png
beetle_311.png
./data/elythra_segmented_full/A00000051187/beetle_311.png A00000051187

KeyboardInterrupt: 

In [ ]:
# i = 0 #df indexer
# print("Segmenting beetle elythras...")
# for fp in image_filepaths:
#     print(fp)
#     image = cv2.imread(fp)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     masks = mask_generator.generate(image)
    
#     #create the path to which the mask will be saved
#     output_path = os.path.join(output_folder, fp.split('/')[-1]) #replace extension and save mask as a png
    
#     os.makedirs(output_folder, exist_ok=True)
    
#     #save mask with cv2 to preserve pixel categories
#     print(f"Mask path:{output_path}")
#     save_anns(masks, image, output_path)


In [ ]:
# i = 0 #df indexer
# print("Segmenting beetle elythras...")
# for fp in image_filepaths:
#     print(fp)
#     # image = cv2.imread(fp)
#     # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     # masks = mask_generator.generate(image)
    
#     # #create the path to which the mask will be saved
#     # output_path = os.path.join(output_folder, fp.split('/')[-1]) #replace extension and save mask as a png
    
#     # os.makedirs(output_folder, exist_ok=True)
    
#     # #save mask with cv2 to preserve pixel categories
#     # print(f"Mask path:{output_path}")
#     # save_anns(masks, image, output_path)

In [ ]:
# masks = mask_generator.generate(image)

In [ ]:
# plt.figure(figsize=(20,20))
# plt.imshow(image)
# show_anns(masks)
# plt.axis('off')
# plt.show() 

In [ ]:


# Note: Ensure crop_to_bb is defined elsewhere in your code.
